In [5]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import eikon as ek
from datetime import timedelta, datetime
import plotly.express as px
import yfinance as yf
import openpyxl

# Working with porfolio dataa

### Creating fake portfolio data

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def create_portfolio_data(start_date, tickers, num_days=20):
    dates = [start_date + timedelta(days=i) for i in range(num_days)]
    data = []

    # Fetch sector information for each ticker
    sector_map = {}
    for ticker in tickers:
        info = yf.Ticker(ticker).info
        sector_map[ticker] = info.get('sector', 'Unknown')

    # Fetch historical market data for each ticker
    historical_data = yf.download(tickers, start=start_date - timedelta(days=1), end=start_date + timedelta(days=num_days), progress=False)

    for date in dates:
        daily_weights = np.random.dirichlet(np.ones(len(tickers)), size=1).flatten()
        for ticker, weight in zip(tickers, daily_weights):
            # Check if date is in the index to avoid KeyError
            if date in historical_data['Adj Close'][ticker].index:
                previous_date = date - timedelta(days=1)
                if previous_date in historical_data['Adj Close'][ticker].index:
                    return_ = historical_data['Adj Close'][ticker].loc[date] - historical_data['Adj Close'][ticker].loc[previous_date]
                    return_ = return_ / historical_data['Adj Close'][ticker].loc[previous_date]
                else:
                    return_ = np.nan  # If no data for previous_date, set return to NaN
            else:
                return_ = np.nan  # If no data for date, set return to NaN
            
            # Use the fetched GICS Sector
            gics_sector = sector_map[ticker]
            data.append([date, ticker, weight, return_, gics_sector])

    return pd.DataFrame(data, columns=['Date', 'Ticker', 'Weight', 'Return', 'GICS Sector'])

# Define the contents of each portfolio
tickers_portfolio_1 = ['AAPL', 'TSLA', 'NVDA', 'GOOGL', 'AMZN']
tickers_portfolio_2 = ['AAPL', 'TSLA', 'NVDA', 'GOOGL', 'AMZN', 'MSFT', 'PLTR', 'XOM', 'VWAGY', 'JPM']

# Create two datasets for the same month with some common tickers
portfolio_1 = create_portfolio_data(datetime(2023, 8, 1), tickers_portfolio_1, num_days=20)
portfolio_2 = create_portfolio_data(datetime(2023, 8, 1), tickers_portfolio_2, num_days=20)

In [7]:
portfolio_1

,Date,Ticker,Weight,Return,GICS Sector
0,2023-08-01,AAPL,0.047994,-0.004276,Technology
1,2023-08-01,TSLA,0.155215,-0.023782,Consumer Cyclical
2,2023-08-01,NVDA,0.107177,-0.004751,Technology
3,2023-08-01,GOOGL,0.590340,-0.008816,Communication Services
4,2023-08-01,AMZN,0.099273,-0.014886,Consumer Cyclical
...,...,...,...,...,...
95,2023-08-20,AAPL,0.314568,NaN,Technology
96,2023-08-20,TSLA,0.227454,NaN,Consumer Cyclical
97,2023-08-20,NVDA,0.033391,NaN,Technology
98,2023-08-20,GOOGL,0.177261,NaN,Communication Services


In [8]:
def merge_dataframes(df1, df2):
    merged_data = (pd.merge(df1, df2, on=['Date', 'Ticker', 'GICS Sector', 'Return'], how='outer', suffixes=('_portofolio', '_benchmark'))
                     .rename(columns={'Weight_portofolio': 'Portfolio Weight', 'Weight_benchmark': 'Benchmark Weight'})
                     .fillna(0)
                   )
    return merged_data

merged_data = merge_dataframes(portfolio_1, portfolio_2)
merged_data.sort_values(by=['Date', 'Ticker'], inplace=True, ignore_index=True)

# Drop dates where the total weighted return for portfolio and benchmark is 0
merged_data

# Remove days when there is no activity in the portfolios
condition = merged_data.groupby('Date')['Return'].transform('sum') != 0
merged_data = merged_data.loc[condition]


### Manipulating the raw data

In [10]:
def calculate_weighted_returns(weights, returns):
    return weights * returns

def calculate_sector_weights(df, weight_column_name):
    return df.groupby(['Date', 'GICS Sector'])[weight_column_name].transform('sum')

# Function to calculate asset weight in sector
def calculate_asset_weight_in_sector(df, weight_column_name):
    sector_weights = calculate_sector_weights(df, weight_column_name)
    
    # If the sector weight is 0, the asset weight in sector is 0. Avoid division by 0.
    return np.where(sector_weights == 0, 0, df[weight_column_name] / sector_weights)

# Function to calculate an assets sector contribution
def calculate_sector_contribution_return(asset_weights_in_sector, returns):
    return asset_weights_in_sector * returns

# Function to calculate daily sector return
def calculate_total_sector_return(df, sector_contribution_column):
    return df.groupby(['Date', 'GICS Sector'])[sector_contribution_column].transform('sum')

# Function to calculate daily portfolio return
def calculate_daily_total_return(df, total_sector_return_column):
    return df.groupby('Date')[total_sector_return_column].transform('sum')

# Function to apply all calculations to a DataFrame for given portfolio or benchmark columns
def apply_calculations_to_df(df, weight_col, return_col, prefix):
    df[f'{prefix} Weighted Return'] = calculate_weighted_returns(df[weight_col], df[return_col])
    df[f'{prefix} Sector Weight'] = calculate_sector_weights(df, weight_col)
    df[f'{prefix} Asset Weight in Sector'] = calculate_asset_weight_in_sector(df, weight_col)
    sector_contribution_return = calculate_sector_contribution_return(df[f'{prefix} Asset Weight in Sector'], df[return_col])
    df[f'{prefix} Sector Contribution Return'] = sector_contribution_return
    total_sector_return = calculate_total_sector_return(df, f'{prefix} Sector Contribution Return')
    df[f'{prefix} Daily Sector Return'] = total_sector_return
    df[f'{prefix} Daily Total Return'] = calculate_daily_total_return(df, f'{prefix} Daily Sector Return')

    return df

# Apply the calculations to both portfolio and benchmark
merged_data = apply_calculations_to_df(merged_data, 'Portfolio Weight', 'Return', 'Portfolio')
merged_data = apply_calculations_to_df(merged_data, 'Benchmark Weight', 'Return', 'Benchmark')

# Attribution calculations

In [11]:
def calculate_allocation_effect(df):
    return (df['Portfolio Sector Weight'] - df['Benchmark Sector Weight']) * (df['Benchmark Daily Sector Return'] - df['Benchmark Daily Total Return'])

def calculate_selection_effect(df):
    return df['Benchmark Sector Weight'] * (df['Portfolio Daily Sector Return'] - df['Benchmark Daily Sector Return'])

def calculate_interaction_effect(df):
    return (df['Portfolio Sector Weight'] - df['Benchmark Sector Weight']) * (df['Portfolio Daily Sector Return'] - df['Benchmark Daily Sector Return'])

def sum_of_effects(allocation, selection, interaction):
    return allocation + selection + interaction

In [12]:
merged_data['Allocation Effect'] = calculate_allocation_effect(merged_data)  
merged_data['Selection Effect'] = calculate_selection_effect(merged_data)
merged_data['Interaction Effect'] = calculate_interaction_effect(merged_data)
merged_data['Sum of Effects'] = sum_of_effects(merged_data['Allocation Effect'], merged_data['Selection Effect'], merged_data['Interaction Effect'])
merged_data['Excess Return'] = merged_data['Portfolio Weighted Return'] - merged_data['Benchmark Weighted Return']
merged_data['Sector Excess Return'] = merged_data['Portfolio Daily Sector Return'] - merged_data['Benchmark Daily Sector Return']

### Demo visualizations

In [13]:
average_sector_weights  = merged_data[['Date', 'GICS Sector', 'Portfolio Sector Weight', 'Benchmark Sector Weight']].drop_duplicates().groupby(['GICS Sector']).mean()

# Now create a Plotly bar chart
fig = px.bar(
    average_sector_weights.reset_index(),  # Reset index to use 'GICS Sector' as a column
    x='GICS Sector',
    y=['Portfolio Sector Weight', 'Benchmark Sector Weight'],
    barmode='group',
    title='Average GICS Sector Weights in Portfolio vs. Benchmark'
)

fig.update_layout(
    template='plotly_dark',
)

# Show the figure
fig



In [14]:
total_effects_per_sector = merged_data[['Date', 'Ticker','GICS Sector', 'Allocation Effect', 'Selection Effect', 'Interaction Effect', 'Sum of Effects', 'Sector Excess Return']].drop_duplicates(subset=['Date', 'GICS Sector', 'Sum of Effects'])

# Take latest day only from total_effects_per_sector
total_effects_per_sector = total_effects_per_sector[total_effects_per_sector['Date'] == total_effects_per_sector['Date'].max()]
total_effects_per_sector


# visualize the total effects per gics sector in a bar chart
fig = px.bar(
    total_effects_per_sector,  # The DataFrame is already filtered, no need to reset the index
    y='GICS Sector',  # Set the y-axis to the 'GICS Sector'
    x=['Allocation Effect', 'Selection Effect', 'Interaction Effect', 'Sum of Effects', 'Sector Excess Return'],  # Ensure column names match
    orientation='h',  # Horizontal bars
    title='Total Effects per GICS Sector on ' + str(total_effects_per_sector['Date'].max().date()),  # Add date to the title
    height=800,  # Set the height of the figure
)

# Improve the layout
fig.update_layout(
    xaxis_title='Total Effects',
    yaxis_title='GICS Sector',
    yaxis=dict(categoryorder='total ascending'),  # Sort sectors by the total effects
    template='plotly_dark'  # Use a dark background for a cleaner look
)

fig


In [15]:
daily_info = merged_data[['Date', 'Allocation Effect', 'Selection Effect', 'Interaction Effect', 'Sum of Effects', 'Sector Excess Return']].drop_duplicates(subset=['Date', 'Sum of Effects']).groupby(['Date']).sum()
daily_info

,Allocation Effect,Selection Effect,Interaction Effect,Sum of Effects,Sector Excess Return
Date,,,,,
2023-08-01,-0.001850,-0.000316,-0.000388,-0.002553,0.007134
2023-08-02,-0.004576,-0.001331,-0.005529,-0.011436,0.007238
2023-08-03,0.001526,0.002252,0.001388,0.005166,-0.014890
2023-08-04,-0.018928,0.008171,-0.005845,-0.016602,0.013703
2023-08-08,0.010260,0.011660,-0.007212,0.014708,0.037194
2023-08-09,-0.003678,0.002978,0.004038,0.003338,0.011055
2023-08-10,-0.000354,0.000052,-0.000221,-0.000523,-0.004497
2023-08-11,-0.005390,-0.004141,0.003463,-0.006068,-0.025500
2023-08-15,0.003684,-0.001151,-0.011001,-0.008467,0.040997


In [16]:
# Function that calculates the compounded return of a series
def calculate_compounded_return(series):
    return np.exp(np.log(series + 1).cumsum()) - 1

daily_excess_return = merged_data[['Date', 'Portfolio Weighted Return', 'Benchmark Weighted Return','Excess Return']].groupby(['Date']).sum()

# Drop values with 0
comp_excess = calculate_compounded_return(daily_excess_return[daily_excess_return['Excess Return'] != 0])

comp_allocation = calculate_compounded_return(daily_info['Allocation Effect'])
comp_selection = calculate_compounded_return(daily_info['Selection Effect'])
comp_interaction = calculate_compounded_return(daily_info['Interaction Effect'])
comp_sum_effects = calculate_compounded_return(daily_info['Sum of Effects'])

compounded_effects = pd.DataFrame({'Excess Return':comp_excess['Excess Return'] ,'Allocation': comp_allocation, 'Selection': comp_selection, 'Interaction': comp_interaction, 'Sum of Effects': comp_sum_effects})

In [17]:
import plotly.express as px

def plot_cumulative_effects(compound_df):
    fig = px.line(
        compound_df,
        title='Cumulative Compounded Returns',
        height=800
    )

    fig.update_layout(
        template='plotly_dark',
        yaxis=dict(tickformat='.2%')
    )

    return fig

plot_cumulative_effects(compounded_effects)


c:\Users\HE1\AppData\Local\miniconda3\envs\data-science\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [18]:
# Plot the portfolio and benchmark sector returns. 
"""
- Compounded returns 

"""
return_df = merged_data[['Date', 'GICS Sector', 'Portfolio Weighted Return', 'Benchmark Weighted Return']].groupby(['GICS Sector', 'Date']).sum()

# Access GICS Sector Communication Services from above
sector_list = return_df.index.get_level_values(0).unique().tolist()
sectors = []

for sector in sector_list:
    benchmark_comp = calculate_compounded_return(return_df.loc[sector].iloc[:,0]).iloc[-1]
    portfolio_comp = calculate_compounded_return(return_df.loc[sector].iloc[:,1]).iloc[-1]
    
    sector_dict = {'Sector':sector, 'Benchmark Sector Return': benchmark_comp, 'Portfolio Sector Return': portfolio_comp}
    sectors.append(sector_dict)

# Turn sectors dict to dataframe
sectors_df = pd.DataFrame(sectors)

fig = px.bar(
    sectors_df,
    x=['Benchmark Sector Return', 'Portfolio Sector Return'],
    y='Sector',
    barmode='group',
    title='Cumulative Compounded Sector Returns',
    height=800
)

fig.update_layout(
    template='plotly_dark',
    xaxis=dict(tickformat='.2%')
)

fig